In [1]:
import pandas as pd
import numpy as np
import os
import tifffile
from lib.fast_dilation_erosion import perform_dilation_erosion

In [2]:
def readfile(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Get the maximum x, y, and z values to determine the shape of the output array
    max_x = int(df['x_in_pix'].max())
    max_y = int(df['y_in_pix'].max())
    max_z = int(df['z_in_pix'].max())

    # Create a new NumPy array with the desired shape, initialized with zeros
    output_array = np.zeros((max_x + 1, max_y + 1, max_z + 1)).astype('int16')

    # Iterate through the DataFrame and store the cell index at the corresponding x, y, z position in the output array
    for index, row in df.iterrows():
        x, y, z = int(row['x_in_pix']), int(row['y_in_pix']), int(row['z_in_pix'])
        cell_index = int(row['Cell Index'])
        output_array[x, y, z] = cell_index

    # Add a padding of 1 layer of zeros around the existing array
    padded_array = np.pad(output_array, pad_width=10, mode='constant', constant_values=0)
    
    return padded_array

In [3]:
os.chdir('/home/akikawa/project/cell_segment_show/')

In [5]:
cell_array = readfile(csv_file = './(2023.5.2)rna_labeled.csv')

In [ ]:
closed_image = perform_dilation_erosion(cell_array, kernel_size=9, iterations=1, border_size=10)

In [ ]:
# Save the closed image as a new 3D TIFF
tifffile.imwrite('./closing_results/output.tif', np.moveaxis(closed_image, 2, 0))